In [97]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#Задание 1.¶
#Обязательная часть
#Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

#Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

#KEYWORDS = ['python', 'парсинг']

#Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

#В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [61]:
URL = 'https://habr.com/ru/all/'
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
req

<Response [200]>

In [193]:
KEYWORDS = ['SolidRun', 'трафик']
habr_news = pd.DataFrame()
row = {}
# извлекаем посты
posts = soup.find_all('article', class_='post')
for post in posts:
    for word in KEYWORDS:
        if word in post.text:
            title_element = post.find('a', class_='post__title_link')
            title_link = title_element.attrs.get('href')
            title_date = post.find('span', class_='post__time')
#            print(title_element.text, title_link, title_date.text)
            row = {'date': title_date.text, 'title': title_element.text, 'link': title_link}
            habr_news = pd.concat([habr_news, pd.DataFrame([row])]) 
habr_news

,date,title,link
0,сегодня в 16:59,Новый CuBox-M: умещающийся на ладони мини-ПК c...,https://habr.com/ru/company/selectel/blog/541998/
0,сегодня в 16:08,ClickHouse: как устроен MergeTree,https://habr.com/ru/post/539538/


In [ ]:
#Задание 2.
#Обязательная часть
#Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
#EMAIL = [xxx@x.ru, yyy@y.com]

#В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

#Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [385]:
URL_AVAST = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
EMAIL = [["oleg@yandex.ru"], ["sov@beepost.ru"]]
avast_dict = {}
params = {

}
headers = {
    'Accept': 'application/json, text/plain, */*',
'Accept-Encoding':'gzip, deflate, br',
'Accept-Language': 'ru,en;q=0.9',
'Connection': 'keep-alive',
'Content-Length': '37',
'Content-Type': 'application/json;charset=UTF-8',
'Host': 'identityprotection.avast.com',
'Origin': 'https://www.avast.com',
'Referer': 'https://www.avast.com/hackcheck/',
'Sec-Fetch-Dest': 'empty',
'Sec-Fetch-Mode': 'cors',
'Sec-Fetch-Site': 'same-site',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 YaBrowser/20.12.3.140 Yowser/2.5 Safari/537.36',
'Vaar-Header-App-Product': 'hackcheck-web-avast',
'Vaar-Version': '0'
}
#перебираем почтовые адреса по списку EMAIL и вставляем их в запрос
for i in range(len(EMAIL)):
    params['emailAddresses'] = EMAIL[i]
    avast = requests.post(URL_AVAST, json = params, headers = headers)
    time.sleep(0.33)
    avast_dict[str(i)] = avast.json()
avast_dict


{'0': {'breaches': {'16372': {'breachId': 16372,
    'site': 'bookmate.com',
    'recordsCount': 7775238,
    'description': 'In July 2018, Bookmate was allegedly breached. The stolen data contains usernames, passwords, salts, email addresses and additional personal information. This data has been sold on at least one dark web market and is also being privately shared among several criminal networks.\r\n',
    'publishDate': '2019-03-20T00:00:00Z',
    'statistics': {'usernames': 7775230,
     'passwords': 2547574,
     'emails': 3822191}},
   '16613': {'breachId': 16613,
    'site': 'verifications.io',
    'recordsCount': 677914246,
    'description': 'Big data e-mail verification platform verifications.io leaked a database containing sensitive PII belonging to over 600 million victims. The breach was discovered by a security researcher who found an unsecured 150GB MongoDB database, which he was able to track to an email verification service called Verifications.io. The leaked databas

In [386]:
avast_dict

{'0': {'breaches': {'16372': {'breachId': 16372,
    'site': 'bookmate.com',
    'recordsCount': 7775238,
    'description': 'In July 2018, Bookmate was allegedly breached. The stolen data contains usernames, passwords, salts, email addresses and additional personal information. This data has been sold on at least one dark web market and is also being privately shared among several criminal networks.\r\n',
    'publishDate': '2019-03-20T00:00:00Z',
    'statistics': {'usernames': 7775230,
     'passwords': 2547574,
     'emails': 3822191}},
   '16613': {'breachId': 16613,
    'site': 'verifications.io',
    'recordsCount': 677914246,
    'description': 'Big data e-mail verification platform verifications.io leaked a database containing sensitive PII belonging to over 600 million victims. The breach was discovered by a security researcher who found an unsecured 150GB MongoDB database, which he was able to track to an email verification service called Verifications.io. The leaked databas

In [387]:
avast.json()

{'breaches': {'17609': {'breachId': 17609,
   'site': 'vk.com',
   'recordsCount': 42536910,
   'description': 'At some time in 2020, the Russian social networking website vKontakte was allegedly breached. The stolen data contains email addresses and additional personal information. This breach is being privately shared on the internet.',
   'publishDate': '2020-05-21T00:00:00Z',
   'statistics': {'usernames': 0, 'passwords': 0, 'emails': 20549631}},
  '16659': {'breachId': 16659,
   'site': 'livejournal.com',
   'recordsCount': 30485836,
   'description': "In 2017, social network LiveJournal's database was allegedly breached. The stolen data contains passwords, email addresses and additional personal information. This breach is being privately shared in the dark web.",
   'publishDate': '2019-05-23T00:00:00Z',
   'statistics': {'usernames': 30485732,
    'passwords': 29634403,
    'emails': 30482018}}},
 'data': {'livejournal.com': {'sov@beepost.ru': [{'breachId': 16659,
     'usernam

In [388]:
avast_df = pd.DataFrame()
#перебираем почтовые адреса по списку EMAIL
for i in range(len(EMAIL)):
    try:
# находим список id инцидентов в ключе 'summary' json словаря и перебираем по нему, если инциденты существуют для данной почты
        for n in range(len(avast_dict[str(i)]['summary'][EMAIL[i][0]]['breaches'])):
#
            id = str(avast_dict[str(i)]['summary'][EMAIL[i][0]]['breaches'][n])
            email = EMAIL[i][0]
            site = avast_dict[str(i)]['breaches'][id]['site']
            date = avast_dict[str(i)]['breaches'][id]['publishDate']
            description = avast_dict[str(i)]['breaches'][id]['description']
            row_avast = {'email': email, 'date': date, 'site': site, 'description' : description}
            avast_df = pd.concat([avast_df, pd.DataFrame([row_avast])])
            
#            print(email, site, date, description)
    except:
        continue
avast_df

,email,date,site,description
0,oleg@yandex.ru,2019-03-20T00:00:00Z,bookmate.com,"In July 2018, Bookmate was allegedly breached...."
0,oleg@yandex.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
0,oleg@yandex.ru,2019-03-07T00:00:00Z,dubsmash.com,"In December 2018, Dubsmash's database was alle..."
0,oleg@yandex.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
0,oleg@yandex.ru,2019-08-08T00:00:00Z,borealis.su,"At an unconfirmed date, Russian Minecraft serv..."
0,oleg@yandex.ru,2020-12-17T00:00:00Z,muslab.ru,"In November 2020, a collection of over 23,000 ..."
0,oleg@yandex.ru,2020-12-17T00:00:00Z,radiocomponent.com.ua,"In November 2020, a collection of over 23,000 ..."
0,oleg@yandex.ru,2020-12-17T00:00:00Z,k4dre.infobox.ru,"In November 2020, a collection of over 23,000 ..."
0,oleg@yandex.ru,2020-12-17T00:00:00Z,rosrealt.ru,"In November 2020, a collection of over 23,000 ..."
0,oleg@yandex.ru,2020-12-03T00:00:00Z,metodika.ru,"In November 2020, a collection of over 23,000 ..."
